In [1]:
!pip install tifffile
!pip install opencv-python
!pip install rasterio
!pip install scipy==1.7.3
!pip install torchvision
!pip install skimage

     ---------------------------------------- 0.0/224.5 kB ? eta -:--:--
     - -------------------------------------- 10.2/224.5 kB ? eta -:--:--
     - -------------------------------------- 10.2/224.5 kB ? eta -:--:--
     - -------------------------------------- 10.2/224.5 kB ? eta -:--:--
     ----- ------------------------------- 30.7/224.5 kB 146.3 kB/s eta 0:00:02
     ---------- -------------------------- 61.4/224.5 kB 234.9 kB/s eta 0:00:01
     ------------------ ----------------- 112.6/224.5 kB 409.6 kB/s eta 0:00:01
     ------------------------------------ 224.5/224.5 kB 723.7 kB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/23.4 MB ? eta -:--:--
      --------------------------------------- 0.3/23.4 MB 6.8 MB/s eta 0:00:04
      --------------------------------------- 0.5/23.4 MB 6.0 MB/s eta 0:00:04
      --------------------------------------- 0.6/23.4 MB 4.6 MB/s eta 0:00:05
     - -------------------------------------- 0.8/23.4 MB 4.5 MB/s eta 0:00:05
     -- ------------------------------------- 1.3/23.4 MB 5.0 MB/s eta 0:00:05
     -- ------------------------------------- 1.7/23.4 MB 5.5 MB/s eta 0:00:04
     --- ------------------------------------ 2.0/23.4 MB 5.9 MB/s eta 0:00:04
     --- ------------------------------------ 2.3/23.4 MB 5.9 MB/s eta 0:00:04
     --- ------------------------------------ 2.3/23.4 MB 5.9 MB/s eta 0:00:04
     ---- ----------------------------------- 2.9/23.4 MB 5.8 MB/s eta 0:00:04
     ----- ---------------------------------- 3.2/23.4 MB 5.8 MB/s eta 0:00:04
     ----- ---------------------------------- 3.5/23.4 MB 

ERROR: Ignored the following versions that require a different python version: 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement scipy==1.7.3 (from versions: 0.8.0, 0.9.0, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.12.1, 0.13.0, 0.13.1, 0.13.2, 0.13.3, 0.14.0, 0.14.1, 0.15.0, 0.15.1, 0.16.0, 0.16.1, 0.17.0, 0.17.1, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.2.1, 1.2.2, 1.2.3, 1.3.0, 1.3.1, 1.3.2, 1.3.3, 1.4.0, 1.4.1, 1.5.0, 1.5.1, 1.5.2, 1.5.3, 1.5.4, 1.6.0, 1.6.1, 1.9.0rc1, 1.9.0rc2, 1.9.0rc3, 1.9.0, 1.9.1, 1.9.2, 1.9.3, 1.10.0rc1, 1.10.0r

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -------------- ------------------------- 0.4/1.2 MB 12.9 MB/s eta 0:00:01
     ------------------------- -------------- 0.7/1.2 MB 9.3 MB/s eta 0:00:01
     ---------------------------------------  1.2/1.2 MB 9.2 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 8.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/198.6 MB ? eta -:--:--
     ---------------------------------------- 0.4/198.6 MB 2.8 MB/s eta 0:01:12
     ---------------------------------------- 0.5/198.6 MB 2.7 MB/s eta 0:01:14
     ---------------------------------------- 1.0/198.6 MB 4.2 MB/s eta 0:00:48
     ---------------------------------------- 1.3/198.6 MB 4.7 MB/s eta 0:00:43
     ---------------------------------------- 1.7/198.6 MB 5.2 MB/s eta 0:00:


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
import pandas as pd
import os
import tifffile
import torch 
import cv2
import matplotlib.pyplot as plt
from torchvision.transforms import ToTensor
import rasterio
from rasterio.plot import show
from PIL import Image
from torchvision.transforms import ToTensor
from utils.data_handlers import link_label1_to_image, link_label0_to_image
from sklearn.model_selection import train_test_split

In [12]:
train_labels_1, train_artifacts_1= link_label1_to_image('train/image', 'train/label2')

In [16]:
all_labels_0, all_artifacts_0=link_label0_to_image('No_Artifacts','No_Artifacts_label')



In [25]:
keys = list(all_labels_0.keys())

# Split the keys into training and testing sets
keys_train, keys_test = train_test_split(keys, test_size=0.2, random_state=42)
train_labels_0 = {key: all_labels_0[key] for key in keys_train}
train_artifacts_0 = {key: all_artifacts_0[key] for key in keys_train}
test_labels_0 = {key: all_labels_0[key] for key in keys_test}
test_artifacts_0 = {key: all_artifacts_0[key] for key in keys_test}


200 51


In [23]:
train_labels_1.update(train_labels_0)
train_labels=train_labels_1
train_artifacts_0.update(train_artifacts_1)
train_artifacts=train_artifacts_0


In [24]:
print(len(train_labels),len(train_artifacts))

347 347


In [28]:
test_labels_1, test_artifacts_1=link_label1_to_image('test/image', 'test/label2')
print(len(test_labels_1), len(test_artifacts_1))

296 296


In [27]:
test_labels_1.update(test_labels_0)
test_labels=test_labels_1
test_artifacts_0.update(test_artifacts_1)
test_artifacts=test_artifacts_0
print(len(test_labels),len(test_artifacts))

347 347
